# 1: Einführung in Pandas

- Umgang mit den grundlegenden Pandas Bausteinen `Series`, `DataFrame`, und `Index`

<div style="text-align: center;">
    <img width="40%" src="https://hips.hearstapps.com/elleuk.cdnds.net/16/36/3200x1599/gallery-1473083573-pandas.jpg" alt="Waving panda" style="min-width: 400px">
    <div><small><em><a href="https://www.elle.com/uk/life-and-culture/culture/news/a31745/10-ways-to-celebrate-pandas-no-longer-being-endangered/">Source</a></em></small></div>
</div>

## Anatomie eines DataFrames

- zweidimensionale Datenstruktur
- aufgebaut auf NumPy Array oder PyArrow

Ein **DataFrame** besteht aus einer oder mehreren **Series**. Die Namen der **Series** sind die Spaltenlabel, und die Zeilenlabel formen den **Index**.

In [ ]:
import pandas as pd

meteorites = pd.read_csv('../data/Meteorite_Landings.csv', nrows=5)
meteorites

*Quelle: [NASA Open Data Portal](https://data.nasa.gov/Space-Science/Meteorite-Landings/gh4g-9sfh)*

In [ ]:
type(meteorites)

#### Series:

In [ ]:
type(meteorites['name'])

In [ ]:
meteorites['name']

#### Columns:

In [ ]:
meteorites.columns

#### Index:

In [ ]:
meteorites.index

## Initialisierung eines DataFrame

Mögliche Datenquellen:
- Python Objekte (dicts, numpy Arrays)
- Textdateien
- Spreadsheets
- Datenbanken
- APIs
- ...

Alle Optionen [hier](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html).

### Textdatei

In [ ]:
import pandas as pd

meteorites = pd.read_csv('../data/Meteorite_Landings.csv')

### API

Herunterladen direkt vom [NASA Open Data Portal](https://data.nasa.gov/Space-Science/Meteorite-Landings/gh4g-9sfh) mit `requests`:

In [ ]:
import requests

response = requests.get(
    'https://data.nasa.gov/resource/gh4g-9sfh.json',
    params={'$limit': 50_000}
)

if response.ok:
    payload = response.json()
else:
    raise ValueError(f'Request was not successful and returned code: {response.status_code}.')

df = pd.DataFrame(payload)
df.head(3)

*Tip: Dann mit `df.to_csv('data.csv')` speichern.*

## Introspektion

#### Wie viele Zeilen und Spalten?

In [ ]:
meteorites.shape

#### Was für Spaltenlabels?

In [ ]:
meteorites.columns

#### Datentypen der Spalten

In [ ]:
meteorites.dtypes

#### Sehen Anfang und Ende schonmal gut aus?

In [ ]:
meteorites.head()

In [ ]:
meteorites.tail()

#### Allgemeine Info im Überblick

In [ ]:
meteorites.info()

### Übung 1.1

##### Erzeuge einen DataFrame aus der Datei `2019_Yellow_Taxi_Trip_Data.csv`. Zeige nur die ersten 3 Zeilen an.

### Übung 1.2

##### Was hat der DataFrame für Dimensionen (Anzahl Spalten und Zeilen)?

## Indexing, Slicing, Filtering

Noch mehr Möglichkeiten als in NumPy.

#### Mehrere Spalten auswählen

In [ ]:
meteorites[['name', 'mass (g)']]

#### Slicing über Zeilen

In [ ]:
meteorites[100:104]

#### Indexing über Position

Analog zu NumPy mit `iloc[]`.

In [ ]:
meteorites.iloc[100:104, [0, 3, 4, 6]]

Oder über die Namen im Index mit `loc[]`:

In [ ]:
meteorites.loc[100:104, 'mass (g)':'year']

#### Filtering mit `bool` Maske

Maske für alle Meteoriten mit Masse über 50g die aufgefunden wurden.

In [ ]:
(meteorites['mass (g)'] > 50) & (meteorites.fall == 'Found')

**Achtung**: Wie bei NumPy müssen die Bedingungen eingeklammert sein, und die bitwise Operatoren (`&`, `|`, `~`) statt der logischen Operatoren (`and`, `or`, `not`) verwendet werden.

In [ ]:
meteorites[(meteorites['mass (g)'] > 1e6) & (meteorites.fall == 'Fell')]

Alternativ kann die `.query()` Methode manchmal leichter lesbar sein.

In [ ]:
meteorites.query("`mass (g)` > 1e6 and fall == 'Fell'")

## Grundlegende Statistiken


#### Wie viele Meteoriten wurden beobachtet und wie viele aufgefunden?

In [ ]:
meteorites.fall.value_counts()

In [ ]:
meteorites.fall.value_counts(normalize=True)

#### Was war die durchschnittliche Masse?

In [ ]:
meteorites['mass (g)'].mean()

Um zu prüfen ob der Mittelwert ein guter Messwert ist, überprüfen wir die Verteilung anhand einiger Quantile.

In [ ]:
meteorites['mass (g)'].quantile([0.01, 0.05, 0.5, 0.95, 0.99])

Bei einer schiefen Verteilung wie dieser ist der Median aussagekräftiger.

In [ ]:
meteorites['mass (g)'].median()

#### Wie schwer war der schwerste Meteorit?

In [ ]:
f"{meteorites['mass (g)'].max() / 1000000} Tonnen"

Daten für diesen Meteoriten:

In [ ]:
meteorites.loc[meteorites['mass (g)'].idxmax()]

*Fun fact: Dieser Meteorit ist in Namibia eine Touristenattraktion.

<div>
    <img width="25%" style="margin-top: -5px; margin-bottom: -5px; min-width: 200px" src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a7/Hoba_meteorite_%2815682150765%29.jpg/1280px-Hoba_meteorite_%2815682150765%29.jpg" alt="Hoba meteorite (source: Wikipedia)"/>
    <div><small><em>Source: Wikipedia</em></small></div>
</div>

#### Wie viele Klassen von Meteoriten sind im Datenset?

In [ ]:
meteorites.recclass.nunique()

Zum Beispiel:

In [ ]:
meteorites.recclass.unique()[:14]

* Mehr Info über Meteoritenklassifikation [hier](https://en.wikipedia.org/wiki/Meteorite_classification)*

#### Statistische Parameter über alle Spalten

In [ ]:
meteorites.describe(include='all')

#### Mehr Möglichkeiten für Statistiken:

- [Series](https://pandas.pydata.org/docs/reference/series.html#computations-descriptive-stats)
- [DataFrame](https://pandas.pydata.org/docs/reference/frame.html#computations-descriptive-stats)

### Übung 1.3

##### Mit den Daten aus `2019_Yellow_Taxi_Trip_Data.csv`, berechne statistische Parameter nur für die Spalten `fare_amount`, `tip_amount`, `tolls_amount`, und `total_amount`.

### Übung 1.4

##### Finde `fare_amount`, `tip_amount`, `tolls_amount`, und `total_amount` für den längsten Trip (nach `trip_distance`).